In [1]:
import numpy as np 
import cv2
import matplotlib.pyplot as plt
import os

In [2]:
def imshow(title = "", image = None, size = 10):
 
 
    plt.figure(figsize=(size , size))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()

In [3]:
def Level_Detection(image):
    img = cv2.imread(image)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to the image to reduce noise
    blurred = cv2.GaussianBlur(gray, (11, 11), 0)

    # Apply thresholding to the blurred image
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Choose the contour with the largest area as the detected bottle
    largest_contour = None
    largest_area = 0
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > largest_area:
            largest_area = area
            largest_contour = contour

    # Draw a bounding box around the detected bottle
    if largest_contour is not None:
        x, y, w, h = cv2.boundingRect(largest_contour)
        #cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.line(img, (x-1500,y), (x+1500, y),(0, 0, 255), 2)

    img = cv2.resize(img,(200,200))



    # Display the resulting image
    imshow('Bottle Detection', img)



In [4]:
def bgSubVid(video):
   
   cap = cv2.VideoCapture(video)

# Get the height and width of the frame (required to be an interfer)
   width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
   height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
   roi_size = (450, 1100)
   roi_center = (int(width/2), int(height/2))
   roi_tl = (int(roi_center[0] - roi_size[0]/12), int(roi_center[1] - roi_size[1]/4))


   kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
   fgbg = cv2.createBackgroundSubtractorKNN()

   while(1):
        ret, frame = cap.read()

        if ret:
          frame_roi = frame[roi_tl[1]:roi_tl[1]+roi_size[1], roi_tl[0]:roi_tl[0]+roi_size[0]]
          fgmask = fgbg.apply(frame_roi)
          fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

          imshow('frame', fgmask)
        else:
           break  

   cap.release()




In [ ]:
cap = cv2.VideoCapture("IMG_0845.mp4")
output_name = 'frame'
output_ext = '.jpg'
frame_count = 0
result_path = "E:\Download\Research_&_Projects\Bottle Filling Project\Frames"

width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
roi_size = (450, 1100)
roi_center = (int(width/2), int(height/2))
roi_tl = (int(roi_center[0] - roi_size[0]/12), int(roi_center[1] - roi_size[1]/4))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_roi = frame[roi_tl[1]:roi_tl[1]+roi_size[1], roi_tl[0]:roi_tl[0]+roi_size[0]]
    cv2.imwrite(os.path.join(result_path,'frame'+str(frame_count)+'.jpg'), frame_roi) 
    frame_count += 1
    cv2.line(frame_roi, (0, 225), (1000, 225), (255, 0, 0), thickness=2)

    for filename in sorted(os.listdir(result_path)):
        if filename.endswith('.jpg'):
        # Read the current image file using cv2.imread()
            img_path = os.path.join(result_path, filename)

    
    Level_Detection(img_path)

    
    
    # Process or display the extracted region as needed
    cv2.imshow('Center Region', frame_roi)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



In [82]:
import cv2
import numpy as np

# Load the video file
cap = cv2.VideoCapture('IMG_0845.mp4')
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
roi_size = (450, 1100)
roi_center = (int(width/2), int(height/2))
roi_tl = (int(roi_center[0] - roi_size[0]/12), int(roi_center[1] - roi_size[1]/4))

kernel = cv2.getStructuringElement(cv2.MORPH_OPEN,(3,3))
fgbg = cv2.createBackgroundSubtractorMOG2()


# Initialize the line color for the liquid level marker
line_color = (0, 0, 255)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break


    frame_roi = frame[roi_tl[1]:roi_tl[1]+roi_size[1], roi_tl[0]:roi_tl[0]+roi_size[0]]
    #cv2.line(frame_roi, (0, 225), (1000, 225), (255, 0, 0), thickness=2)
    fgmask = fgbg.apply(frame_roi)

    color_fg = cv2.bitwise_and(frame_roi, frame_roi, mask=fgmask)

    # Convert the frame to grayscale and apply a Gaussian blur
    gray = cv2.cvtColor(frame_roi, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)


    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # Apply Canny edge detection to detect edges
    edges = cv2.Canny(thresh, 700, 800)

    # Perform Hough transform to detect lines
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=56, minLineLength=40, maxLineGap=9)

    # Filter out any lines that are not part of the bottle or the liquid surface
    bottle_lines = []
    liquid_lines = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if abs(y2 - y1) < 5:
            bottle_lines.append(line)
        else:
            liquid_lines.append(line)

    # Find the intersection point of the liquid lines to estimate the liquid level
    liquid_points = []
    for line1 in liquid_lines:
        for line2 in liquid_lines:
            if line1 is not line2:
                x1, y1, x2, y2 = line1[0]
                x3, y3, x4, y4 = line2[0]
                if abs(y2 - y1) > 0 and abs(y4 - y3) > 0:
                    m1 = (x2 - x1) / (y2 - y1)
                    m2 = (x4 - x3) / (y4 - y3)
                    if abs(m1 - m2) > 0:
                        b1 = x1 - m1 * y1
                        b2 = x3 - m2 * y3
                        x = int((b2 - b1) / (m1 - m2))
                        y = int(m1 * x + b1)
                        liquid_points.append((x, y))

    # Draw the detected lines on the frame
    for line in bottle_lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(color_fg, (x2, y2), (x1, y1), (0, 0, 255), 2)
    for line in liquid_lines:
        x1, y1, x2, y2 = line[0]
        #cv2.line(color_fg, (x1,y2), (x2, y1), (255, 0, 0), 2)

    # Draw the estimated liquid level marker on the frame
    if len(liquid_points) > 0:
        x, y = liquid_points[0]
        #cv2.line(color_fg, (0, y), (frame_roi.shape[1], y), line_color, 2)

    # Show the resulting frame
    cv2.imshow('frame', color_fg)
    

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [84]:
import cv2
import numpy as np

cap = cv2.VideoCapture('IMG_0845.mp4')
cap = cv2.VideoCapture('IMG_0845.mp4')
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
roi_size = (450, 1100)
roi_center = (int(width/2), int(height/2))
roi_tl = (int(roi_center[0] - roi_size[0]/12), int(roi_center[1] - roi_size[1]/4))

kernel = cv2.getStructuringElement(cv2.MORPH_OPEN,(3,3))
fgbg = cv2.createBackgroundSubtractorMOG2()

while(cap.isOpened()):
    ret, frame = cap.read()
    frame_roi = frame[roi_tl[1]:roi_tl[1]+roi_size[1], roi_tl[0]:roi_tl[0]+roi_size[0]]

    fgmask = fgbg.apply(frame_roi)

    color_fg = cv2.bitwise_and(frame_roi, frame_roi, mask=fgmask)

    gray = cv2.cvtColor(frame_roi, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    ret, threshold = cv2.threshold(edges, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    contours, hierarchy = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_sizes = [(cv2.contourArea(contour), contour) for contour in contours]
    if contour_sizes:
        max_contour = max(contour_sizes, key=lambda x: x[0])[1]
        (x, y), radius = cv2.minEnclosingCircle(max_contour)
        center = (int(x), int(y))
        radius = int(radius)
        bottle_height = 250  # height of the bottle in pixels
        liquid_level = int(bottle_height - radius - y)
        cv2.circle(color_fg, center, radius, (0, 0, 255), 2)
        cv2.putText(color_fg, f'Liquid Level: {liquid_level}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.imshow('frame', color_fg)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
